# 07.3 Martingales in a Jump–Diffusion Model

<h3><span style="color:#800000;"><strong>Authored by:</strong> <em>Alexandre Mathias DONNAT, Sr</em></span></h3>

**Goal of this notebook**:
- Understand the martingale structure of jump–diffusion models:
- how drift, compensation, and jumps interact so that the discounted price becomes a martingale under a risk-neutral measure.
- Core step that allows risk-neutral pricing and the hedging formulas of notebook 07.4.

**This notebook**: 

Inside, we derive the martingale condition for a jump–diffusion, introduce compensated Poisson integrals, prove the discounted asset is a martingale, and verify the property numerically through simulation.

# 1. Reminder of imposed setting

We consider:

- Brownian motion $W_t$,
- Poisson process $N_t$ with intensity $\lambda$,
- i.i.d. jump amplitudes $U_j$ with distribution $\nu$,
- All objects mutually independent.

**The risky asset:**

$$dX_t = \mu X_{t-} dt + \sigma X_{t-} dW_t + X_{t-} dZ_t, \quad Z_t = \sum_{j=1}^{N_t} U_j.$$

**Riskless asset:** $S_t^0 = e^{rt}$.

**Discounted asset:**

$$\tilde{X}_t = e^{-rt} X_t.$$


# 2. Martingale Condition

We want:

$$\tilde{X}_t = e^{-rt} X_t \text{ is a martingale.}$$

Under the integrability condition $\mathbb{E}|U_1| < \infty$,

$$\tilde{X}_t \text{ is a martingale } \iff \mu = r - \lambda \mathbb{E}[U_1].$$

That is to say :
expected jumps add a drift, so the diffusion drift must compensate it.


# 3. Compensated Poisson Integral

Define the compensated process:

$$\tilde{N}_t = N_t - \lambda t$$

Then $\tilde{N}_t$ is a martingale.

More generally, for measurable $\Phi(y,z)$:

$$M_t = \sum_{j=1}^{N_t} \Phi(Y_{\tau_j}, U_j) - \lambda \int_0^t ds \int \Phi(Y_s, z) \nu(dz)$$

is a square-integrable martingale.

This is the jump analogue of stochastic integration w.r.t. Brownian motion.


# 4. Independence Structure

At any stopping time $s$:

- Brownian increments $W_t - W_s$,
- Poisson increments $N_t - N_s$,
- Future jump amplitudes $U_{N_s+1}, U_{N_s+2}, \ldots$,

are independent of $\mathcal{F}_s$.

This property is crucial for martingale decompositions and for risk-neutral pricing under jumps.


# 5. Discounted Dynamics

Under the martingale drift condition:

$$\mu = r - \lambda \mathbb{E}[U_1],$$

the discounted price satisfies:

$$d\tilde{X}_t = \tilde{X}_{t-} \left[\sigma \, dW_t + \left(dZ_t - \lambda \mathbb{E}[U_1] \, dt\right)\right].$$

This is a sum of:

- a Brownian martingale term,
- a compensated Poisson martingale term.

Hence:

$$\tilde{X}_t \text{ is a true martingale.}$$


# 6. Numerical Verification by simulation

Below is a small simulation confirming that

$$\mathbb{E}[\tilde{X}_t] \approx X_0$$

when the drift satisfies the martingale condition.


In [3]:
import numpy as np

np.random.seed(0)

# Parameters
T = 1.0
n_steps = 500
dt = T / n_steps
Npaths = 20000

r = 0.02
sigma = 0.25
lam = 1.2
mu_jump = -0.1              # mean jump size E[U]
U_std = 0.15                # jump amplitude std

# Martingale drift condition
mu = r - lam * mu_jump

# Simulation containers
X = np.full(Npaths, 100.0)
discount = np.exp(-r * dt)

for _ in range(n_steps):
    # Brownian increment
    dW = np.sqrt(dt) * np.random.randn(Npaths)

    # Poisson jumps: number of jumps = 0 or 1 most of the time
    N_jump = np.random.poisson(lam * dt, size=Npaths)
    
    # Jump amplitudes (Gaussian example)
    U = np.where(N_jump > 0,
                 np.random.normal(mu_jump, U_std, size=Npaths),
                 0.0)
    
    # Update process
    X = X * (1 + mu * dt + sigma * dW + U)

# Discounted terminal value
X_tilde = np.exp(-r * T) * X

print("Mean discounted value:", X_tilde.mean())


Mean discounted value: 100.07285495608296


The discounted terminal value is essentially equal to the initial price $X_0 = 100$, confirming through simulation that the process is a martingale when the drift satisfies the jump–diffusion martingale condition.